# BioKnowledge reviewer implementation

The following code uses the bioknowledge reviewer framework to retrieve all necessary data to create the knowledge graphs

In [2]:
import transcriptomics, regulation, curation, monarch, graph, neo4jlib, hypothesis, summary, utils
import pandas as pd

C:\Users\mirei\Anaconda3\lib\site-packages\google\auth\__init__.py:55: Python37DeprecationWarning: After January 1, 2024, new releases of this library will drop support for Python 3.7. More details about Python 3.7 support can be found at https://cloud.google.com/python/docs/python37-sunset/
  warnings.warn(message, Python37DeprecationWarning)
C:\Users\mirei\Anaconda3\lib\site-packages\google\oauth2\__init__.py:40: Python37DeprecationWarning: After January 1, 2024, new releases of this library will drop support for Python 3.7. More details about Python 3.7 support can be found at https://cloud.google.com/python/docs/python37-sunset/
  warnings.warn(message, Python37DeprecationWarning)


## Subgraphs

### Curation

In [ ]:
curation_edges = pd.read_csv("curation/data/HD/HD_curated_edges.csv")
curation_nodes = pd.read_csv("curation/data/HD/HD_curated_nodes.csv")

### Transcriptomics

In [ ]:
%%time
# transcriptomics:
csv_path = './transcriptomics/GSE64810_mlhd_DESeq2_diffexp_DESeq2_outlier_trimmed_adjust.txt'
data = transcriptomics.read_data(csv_path, sep='\t')
clean_data, up, down = transcriptomics.clean_data(data)
# clean_data()" is running. Keeping only data with FC > 1.5 and FDR < 5%
data_edges = transcriptomics.prepare_data_edges(up, down) ###
rna_network = transcriptomics.prepare_rna_edges(data_edges)

# build network with graph schema
rna_edges = transcriptomics.build_edges(rna_network)
rna_nodes, node_dict = transcriptomics.build_nodes(rna_network)
rna_edges = transcriptomics.rework_edges(pd.DataFrame(rna_edges), node_dict)

### Regulation

In [ ]:
%%time
# prepare msigdb data
gmt_path = './regulation/msigdb/data/c3.tft.v6.1.entrez.gmt'
regulation.prepare_msigdb_data(gmt_path)

# prepare individual networks
data = regulation.load_tf_gene_edges()
dicts = regulation.get_gene_id_normalization_dictionaries(data)
data_edges = regulation.prepare_data_edges(data, dicts)

# prepare regulation network
reg_network = regulation.prepare_regulation_edges(data_edges)

# build network with graph schema
reg_edges = regulation.build_edges(reg_network)
reg_nodes = regulation.build_nodes(reg_network)

### Monarch

In [3]:
%%time
# prepare data to graph schema
# seed nodes
seedList_a = [
    'MONDO:0004975', #AD
    'HGNC:620', #APP
    'HGNC:2095', #Clu
    'HGNC:6893', #MAPT
    'HGNC:933', #BACE1
    'HGNC:613' #APOE
]

# alz 'HGNC:9508' = #PSEN1, 'HGNC:9509' = #PSEN2

seedList = [ 
    'MONDO:0007739', # HD
    'HGNC:4851' # HTT
] 

# get first shell of neighbours
neighboursList = monarch.get_neighbours_list(seedList)
print(len(neighboursList))

# introduce animal model ortho-phenotypes for seed and 1st shell neighbors
## For seed nodes:
seed_orthophenoList = monarch.get_orthopheno_list(seedList)
print(len(seed_orthophenoList))
## For 1st shell nodes:
neighbours_orthophenoList = monarch.get_orthopheno_list(neighboursList)
print(len(neighbours_orthophenoList))

# network nodes: seed + 1shell + ortholog-phentoype
geneList = sum([seedList,
                neighboursList,
                seed_orthophenoList,
                neighbours_orthophenoList
               ], 
               [])
print('genelist: ',len(geneList))

# get Monarch network
monarch_network = monarch.extract_edges(geneList)
print('network: ',len(monarch_network))

# save edges
monarch.print_network(monarch_network, 'monarch_connections')

# build network with graph schema 
monarch_edges = monarch.build_edges(monarch_network)
monarch_nodes = monarch.build_nodes(monarch_network)


The function "get_neighbours_list()" is running. Its runtime may take some minutes. If you interrupt the process, you will lose all the nodes retrieved and you should start over the execution of this function.


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.66s/it]



Finished get_neighbours_list().

419

The function "get_orthopheno_list()" is running. Its runtime may take some hours. If you interrupt the process, you will lose all the nodes retrieved and you should start over the execution of this function.


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.87s/it]
0it [00:00, ?it/s]



Finished get_orthopheno_list().

0

The function "get_orthopheno_list()" is running. Its runtime may take some hours. If you interrupt the process, you will lose all the nodes retrieved and you should start over the execution of this function.


100%|████████████████████████████████████████████████████████████████████████████████| 419/419 [18:40<00:00,  2.68s/it]
0it [00:00, ?it/s]



Finished get_orthopheno_list().

0
genelist:  421

The function "extract_edges()" is running. Its runtime may take some hours. If you interrupt the process, you will lose all the edges retrieved and you should start over the execution of this function.


100%|████████████████████████████████████████████████████████████████████████████████| 421/421 [19:40<00:00,  2.80s/it]



Finished extract_edges(). To save the retrieved Monarch edges use the function "print_network()".

network:  10597

Saving Monarch edges at: 'C:\Users\mirei\Desktop\LEIDEN\th\kg\bioknowledge-reviewer-ADHD\bioknowledge_reviewer/monarch/monarch_connections_v2024-05-27.csv'...


The function "build_edges()" is running...
df (10597, 9)

* This is the size of the edges file data structure: (10597, 9)
* These are the edges attributes: Index(['subject_id', 'object_id', 'property_id', 'property_label',
       'property_description', 'property_uri', 'reference_uri',
       'reference_supporting_text', 'reference_date'],
      dtype='object')
* This is the first record:
  subject_id  object_id             property_id          property_label  \
0  HGNC:3272  HGNC:3274  biolink:interacts_with  biolink:interacts_with   

  property_description                                       property_uri  \
0                   NA  http://purl.obolibrary.org/obo/biolink_interac...   

                        

## Merge

In [4]:
#edges
curation_edges = "curation/data/Empty_edges.csv"
monarch_edges = "monarch/monarch_edges_v2024-05-27.csv"
regulation_edges = "graph/regulation_edges_v2024-05-25.csv"
transcriptomics_edges = "curation/data/Empty_edges.csv"  #############

# nodes
curation_nodes = "curation/data/Empty_nodes.csv"
monarch_nodes = "monarch/monarch_nodes_v2024-05-27.csv"
regulation_nodes = "graph/regulation_nodes_v2024-05-25.csv"
transcriptomics_nodes = "curation/data/Empty_nodes.csv" ###########

In [5]:
# EXTRA CONNECTIVITY FOR TRANSCR+MONARCH

# Monarch graph connectivity
graph_nodes_list, reg_graph_edges = graph.graph_nodes(
    curation=pd.read_csv(curation_edges),
    monarch=pd.read_csv(monarch_edges),
    transcriptomics=pd.read_csv(transcriptomics_edges),
    regulation=pd.read_csv(regulation_edges)
)

C:\Users\mirei\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3417: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)



The function "graph_nodes()" is running...

Preparing networks...
Curated:
(0, 9)
Index(['subject_id', 'property_id', 'object_id', 'reference_uri',
       'reference_supporting_text', 'reference_date', 'property_label',
       'property_description', 'property_uri'],
      dtype='object')
Monarch:
(10597, 9)
Index(['subject_id', 'property_id', 'object_id', 'reference_uri',
       'reference_supporting_text', 'reference_date', 'property_label',
       'property_description', 'property_uri'],
      dtype='object')
Transcriptomics:
(0, 9)
Index(['subject_id', 'property_id', 'object_id', 'reference_uri',
       'reference_supporting_text', 'reference_date', 'property_label',
       'property_description', 'property_uri'],
      dtype='object')
Regulatory:
(197267, 9)
Index(['subject_id', 'object_id', 'property_id', 'property_label',
       'property_description', 'property_uri', 'reference_uri',
       'reference_supporting_text', 'reference_date'],
      dtype='object')

Concatenating in

In [6]:
monarch_network_graph = monarch.extract_edges(graph_nodes_list)
print('network: ',len(monarch_network_graph))


The function "extract_edges()" is running. Its runtime may take some hours. If you interrupt the process, you will lose all the edges retrieved and you should start over the execution of this function.


  0%|                                                                                        | 0/17084 [00:00<?, ?it/s]

error: <class 'AttributeError'>


  0%|                                                                             | 18/17084 [00:30<8:13:46,  1.74s/it]

error: <class 'IndexError'>


...

error: <class 'IndexError'>


100%|████████████████████████████████████████████████████████████████████████▉| 17075/17084 [10:44:15<00:12,  1.39s/it]

error: <class 'IndexError'>


100%|█████████████████████████████████████████████████████████████████████████| 17084/17084 [10:44:25<00:00,  2.26s/it]


Finished extract_edges(). To save the retrieved Monarch edges use the function "print_network()".

network:  1159364


In [7]:
## save Monarch network
monarch.print_network(monarch_network_graph, 'monarch_connections_graph')

## build Monarch network with graph schema
monarch_graph_edges = monarch.build_edges(monarch_network_graph)
monarch_graph_nodes = monarch.build_nodes(monarch_network_graph)


Saving Monarch edges at: 'C:\Users\mirei\Desktop\LEIDEN\th\kg\bioknowledge-reviewer-ADHD\bioknowledge_reviewer/monarch/monarch_connections_graph_v2024-05-27.csv'...


The function "build_edges()" is running...
df (1159364, 9)

* This is the size of the edges file data structure: (1159364, 9)
* These are the edges attributes: Index(['subject_id', 'object_id', 'property_id', 'property_label',
       'property_description', 'property_uri', 'reference_uri',
       'reference_supporting_text', 'reference_date'],
      dtype='object')
* This is the first record:
   subject_id   object_id      property_id   property_label  \
0  HGNC:27414  GO:0005515  biolink:enables  biolink:enables   

  property_description                                    property_uri  \
0                   NA  http://purl.obolibrary.org/obo/biolink_enables   

  reference_uri                          reference_supporting_text  \
0            NA  This edge comes from the Monarch Knowledge Gra...   

  reference_date  


In [9]:
# build review graph
monarch_graph_edges = "monarch/monarch_edges_v2024-05-27.csv"
monarch_graph_nodes = "monarch/monarch_nodes_v2024-05-27.csv"

edges = graph.build_edges(
    curation=curation_edges,
    monarch=monarch_graph_edges, 
    regulation=regulation_edges,
    transcriptomics=transcriptomics_edges, 
    input_from_file=True
)
nodes = graph.build_nodes(
    statements= edges,
    curation=curation_nodes,
    monarch=monarch_graph_nodes, 
    regulation=regulation_nodes,
    transcriptomics=transcriptomics_nodes, 
    input_from_file=True
)


The function "build_edges()" is running...

Preparing networks...
Curated:
(0, 9)
Index(['subject_id', 'property_id', 'object_id', 'reference_uri',
       'reference_supporting_text', 'reference_date', 'property_label',
       'property_description', 'property_uri'],
      dtype='object')
Monarch:
(1159364, 9)
Index(['subject_id', 'property_id', 'object_id', 'reference_uri',
       'reference_supporting_text', 'reference_date', 'property_label',
       'property_description', 'property_uri'],
      dtype='object')
Transcriptomics:
(0, 9)
Index(['subject_id', 'property_id', 'object_id', 'reference_uri',
       'reference_supporting_text', 'reference_date', 'property_label',
       'property_description', 'property_uri'],
      dtype='object')
Regulatory:
(197267, 9)
Index(['subject_id', 'object_id', 'property_id', 'property_label',
       'property_description', 'property_uri', 'reference_uri',
       'reference_supporting_text', 'reference_date'],
      dtype='object')

Concatenating 

In [ ]:
# import to graph database
## prepare the graph to neo4j format
edges_df = utils.get_dataframe(edges)
nodes_df = utils.get_dataframe(nodes)
statements = neo4jlib.get_statements(edges_df)
concepts = neo4jlib.get_concepts(nodes_df)
print('statements: ', len(statements))
print('concepts: ',len(concepts))

## save files into neo4j import dir
neo4j_path = ''#'./{}'.format(neo4j_dir)
neo4jlib.save_neo4j_files(statements, neo4j_path, file_type = 'statements')
neo4jlib.save_neo4j_files(concepts, neo4j_path, file_type = 'concepts')